In [13]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
# import scipy.sparse as sp

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from models import NCF

# -----------------------------------------------------------------

In [1]:
from torch.utils.data import DataLoader
from data import DataGenerator

data = DataGenerator()

# -----------------------------------------------------------------

In [2]:
batch_size = 2048
dataloader = DataLoader(data, batch_size=batch_size,
                        shuffle=True, num_workers=0)

# -----------------------------------------------------------------

In [4]:
'''TRAIN MODEL ( MINI BATCH )'''

def evaluate_model(model, df_val:pd.DataFrame, top_K, random_samples):
    model.eval()
    avg_HR = np.zeros((len(df_val), top_K))
    avg_NDCG = np.zeros((len(df_val), top_K))
    test_df = data.add_negatives(
        df_val,
        n_samples=random_samples
    )
    gp = test_df.groupby('uid')
    for g in gp:
        for k in range(top_K):
            users, items = torch.LongTensor(g.uid).to(device), torch.LongTensor(g.mid).to(device)
            y_hat = model(users, items)
            y_hat = y_hat.cpu().detach().numpy().reshape((-1,))
            test_item_input = items.cpu().detach().numpy().reshape((-1,))
            map_item_score = dict(zip(test_item_input, y_hat))

    for i in range(df_val.shape[0]):
        for k in range(top_K):
            test_df = data.add_negatives(
                pd.DataFrame(data.test.iloc[i]).T,
                n_samples=random_samples
            )
            users, items = torch.LongTensor(test_df.uid).to(device), torch.LongTensor(test_df.mid).to(device)
            y_hat = model(users, items)
            y_hat = y_hat.cpu().detach().numpy().reshape((-1,))
            test_item_input = items.cpu().detach().numpy().reshape((-1,))
            map_item_score = dict(zip(test_item_input, y_hat))
            # Evaluate top rank list
            ranklist = heapq.nlargest(k, map_item_score, key=map_item_score.get)
            gtItem = test_item_input[0]
            for item in ranklist:
                if item==gtItem:
                    avg_HR[i, k] = 1
                    avg_NDCG[i, k] = math.log(2) / math.log(i+2)
                else:
                    avg_HR[i, k] = 0
                    avg_NDCG[i, k] = 0
    avg_HR = np.mean(avg_HR, axis=0)
    avg_NDCG = np.mean(avg_NDCG, axis=0)
    return avg_HR, avg_NDCG

def evaluate(model, df_val:pd.DataFrame, k=10):
    test_df = data.add_negatives(df_val, n_samples=random_samples)
    users, items = torch.LongTensor(test_df.uid).to(device), torch.LongTensor(test_df.mid).to(device)
    y_hat = model(users, items)
    test_df['score'] = y_hat.detach.numpy().reshape((-1,))
    grouped = test_df.copy(deep=True)
    grouped['ranked'] = grouped.groupby('uid')['score'].rank(method='first', ascending=False)
    grouped.sort_values(['uid', 'rank'], inplace=True)
    top_k = grouped[grouped['rank']<=k]
    test_in_top_k = top_k[top_k['rating'] == 1]
    hr = test_in_top_k.shape[0] / data.num_users
    test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: np.log(2)/np.log(1 + x))
    ndcg = test_in_top_k.ndcg.sum() / data.num_users
    return hr, ndcg

def get_test_instances_with_random_samples(data, random_samples, num_items, device):
    user_input = np.zeros((random_samples + 1))
    item_input = np.zeros((random_samples + 1))

    # positive instance
    user_input[0] = data[0]
    item_input[0] = data[1]
    i = 1
    # negative instances
    checkList = data[1]
    for t in range(random_samples):
        j = np.random.randint(num_items)
        while j == checkList:
            j = np.random.randint(num_items)
        user_input[i] = data[0]
        item_input[i] = j
        i += 1
    return torch.LongTensor(user_input).to(device), torch.LongTensor(item_input).to(device)

def evaluate_model(model, df_val, top_K, random_samples, num_items):
    model.eval()
    avg_HR = np.zeros((len(df_val), top_K))
    avg_NDCG = np.zeros((len(df_val), top_K))

    for i in range(len(df_val)):
        test_user_input, test_item_input = get_test_instances_with_random_samples(df_val[i], random_samples, num_items,
                                                                                  device)
        y_hat = model(test_user_input, test_item_input)
        y_hat = y_hat.cpu().detach().numpy().reshape((-1,))
        test_item_input = test_item_input.cpu().detach().numpy().reshape((-1,))
        map_item_score = {}
        for j in range(len(y_hat)):
            map_item_score[test_item_input[j]] = y_hat[j]
        for k in range(top_K):
            # Evaluate top rank list
            ranklist = heapq.nlargest(k, map_item_score, key=map_item_score.get)
            gtItem = test_item_input[0]
            avg_HR[i, k] = getHitRatio(ranklist, gtItem)
            avg_NDCG[i, k] = getNDCG(ranklist, gtItem)
    avg_HR = np.mean(avg_HR, axis=0)
    avg_NDCG = np.mean(avg_NDCG, axis=0)
    return avg_HR, avg_NDCG

num_epochs = 25
learning_rate = .001

emb_size = 128
hidden_layers = np.array([emb_size, 64, 32, 16])
output_size = 1
random_samples = 100
num_negatives = 4
top_K = 10
# -----------------------------------------------------------------

NameError: name 'pd' is not defined

In [ ]:
def train_model(model):
    # data.get_train_instances(seed=e)
    optimizer = torch.optim.AdamW(model.parameters(),  lr=learning_rate, weight_decay=1e-6)

    dataloader = DataLoader(data, batch_size=batch_size,
                        shuffle=True, num_workers=0)
    t1 = time.time()

    it_per_epoch = len(data) / batch_size

    for i in range(num_epochs):
        print("Starting epoch ", i + 1)
        j = 0
        for batch in dataloader:
            u, m, r = batch
            # move tensors to cuda
            u = u.to(device)
            m = m.to(device)
            r = r.to(device)

            y_hat = model(u, m)

            loss = torch.nn.BCELoss()  # (weight=w, reduction="mean")

            loss = loss(y_hat, r.unsqueeze(1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if i % int(1 + it_per_epoch / 10) == 0:
                print("Progress: ", round(100 * j / it_per_epoch), "%")

        # Epoch metrics
        t2 = time.time()
        print("Epoch time:", round(t2 - t1), "seconds")
        print("Loss:", loss / i)
        print("Evaluating model...")
        # new
        # HR, NDCG = evaluate_model(model, data, validation=False)
        # updated
        # hr, ndcg = evaluate_model(model, data.test, top_K, random_samples)
        # original
        ht, ndcg = evaluate_model(
            NCF,
            data.test.values,
            25,
            random_samples,
            data.num_movies
        )
        print("HR@i:", hr)
        print("NDCG@i", ndcg)
        loss = 0
        print()

    print("Done")

import random
import copy
import utils

def fed_fit(model_central, data, C, batch_size, epochs, lr, eta, verbose=True):

    # Sample the participants for the round of training
    num_participants = int(data.num_users * C)
    participants = random.sample(range(data.num_users), num_participants)

    # model_difference holds the total change of the global model after the round
    model_difference = copy.deepcopy(model_central)
    utils.zero_model_parameters(model_difference)

    it = 0

    t1 = time.time()

    # Start training loop
    for user in participants:

        it += 1
        if it % int(num_participants / 10) == 0 and verbose:
            print("Progress:", round(100 * it / num_participants), "%")

        # The current user takes a copy of the global model
        model_client = copy.deepcopy(model_central)

        # Defining optimizers
        optimizer = torch.optim.SGD(model_client.parameters(), lr=lr)  # MLP optimizer
        optimizer_u = torch.optim.SGD(model_client.user_embedding.parameters(), lr=lr / C * eta - lr)  # User optimizer
        optimizer_i = torch.optim.SGD(model_client.item_embedding.parameters(),
                                      lr=lr * data.num_items * eta - lr)  # Item optimizer

        # Prepares data for the current user
        # data.set_current_user(user)
        # data.generate_negatives()

        dataloader = DataLoader(data, batch_size=batch_size,
                                shuffle=True, num_workers=0)

        # Trains on the users data
        for e in range(epochs):
            for batch in dataloader:
                # Load tensors of users, movies, outputs and loss weights
                u, m, y = batch
                # move tensors to cuda
                u = u.to(device)
                m = m.to(device)
                y = y.to(device)
                # w = w.to(device)

                # make predictions
                p_pred = model_client(u, m)

                # Calculate mean loss
                loss_fn = torch.nn.BCELoss()  # weight=w, reduction="mean")
                loss = loss_fn(p_pred, y)

                # Backpropagate the output and update model parameters
                optimizer.zero_grad()
                optimizer_u.zero_grad()
                optimizer_i.zero_grad()

                loss.backward()
                optimizer.step()
                optimizer_u.step()
                optimizer_i.step()

        # Calculate the user's change of the model and add it to the total change
        utils.sub_model_parameters(model_central, model_client)
        utils.add_model_parameters(model_client, model_difference)

    # Take the average of the MLP and item vectors
    utils.divide_model_parameters(model_difference, num_participants)

    # Update the global model by adding the total change
    utils.add_model_parameters(model_difference, model_central)
    t2 = time.time()
    print("Time of round:", round(t2 - t1), "seconds")


# -----------------------------------------------------------------
# --- Train FedFit model  ---

# Hyper parameters
C = 0.4  # pop sample ratio
eta = 80  # federated param
E = 1  # epochs
# B = 102  # batch size
T = 196  # num rounds
# lr = 0.3  # learning rate

# Initiate model
model_central = NCF(num_users=data.num_users, num_items=data.num_movies, embed_size=hidden_layers, output_size=output_size)

print("Processing data...")
data = DataGenerator()
print("Done")

for t in range(T):  # for each round
    print("Starting round", t + 1)
    # train one round
    fed_fit(model_central, data, C=C, batch_size=batch_size, epochs=E, lr=learning_rate, eta=eta, verbose=True)

    print("Evaluating model...")
    HR, NDCG = evaluate_model(model_central, data, validation=False)
    print("HR@10:", HR)
    print("NDCG@10", NDCG)

torch.save(model_central.state_dict(), "models/federatedNCF")

# -----------------------------------------------------------------

In [ ]:
ncf = NCF(data.num_users, data.num_movies, emb_size, hidden_layers, output_size).to(device)

train_model(ncf)

torch.save(ncf.state_dict(), "models/preTrained_NCF")

# -----------------------------------------------------------------

In [ ]:
''' GET USER EMBEDDING '''
from data import FairnessData

sensitive = FairnessData()
clean = sensitive.df

train_ratio = 0.7

train, test = sensitive.train_test_split(train_ratio)


clean.info()  # 1476+3444 = 4920

In [ ]:
user_embeds = ncf.user_emb.weight.data.cpu().detach().numpy()
user_embeds = user_embeds.astype('float')

# -----------------------------------------------------------------

In [ ]:
''' COMPUTE GENDER EMBEDDING '''
gender_embed = np.zeros((2,user_embeds.shape[1]))
num_users_x_group = np.zeros((2, 1))

for i in range(train.shape[0]):
    u = train['uid'].iloc[i]
    if train['gender'].iloc[i] == 0:
        gender_embed[0] +=  user_embeds[u]
        num_users_x_group[0] += 1.0
    else:
        gender_embed[1] +=  user_embeds[u]
        gender_embed[1] += 1.0
        num_users_x_group[1] += 1.0
# -----------------------------------------------------------------

In [ ]:
''' VERTICAL BIAS'''
gender_embed = gender_embed / num_users_x_group
# vBias = compute_bias_direction(gender_embed)
vBias = gender_embed[1].reshape((1,-1)) - gender_embed[0].reshape((1,-1))
vBias = vBias / np.linalg.norm(vBias,axis=1,keepdims=1)

vBias

# -----------------------------------------------------------------

In [ ]:
''' LINEAR PROJECTION '''

# debias users
debiased_user_embeds = user_embeds
for i in range(len(clean)):
    u = clean['uid'].iloc[i]
    debiased_user_embeds[u] = user_embeds[u] - (np.inner(user_embeds[u].reshape(1,-1),vBias)[0][0])*vBias

# -----------------------------------------------------------------

In [ ]:
'''UPDATE USER EMBEDDINGS'''

fairness_thres = torch.tensor(0.1).to(device)
epsilonBase = torch.tensor(0.0).to(device)

n_careers = sensitive.num_jobs

# replace page items with career items
ncf.like_emb = nn.Embedding(n_careers,emb_size).to(device)
# freeze user embedding
ncf.user_emb.weight.requires_grad=False

# replace user embedding of the model with debiased embeddings
ncf.user_emb.weight.data = torch.from_numpy(debiased_user_embeds.astype(np.float32)).to(device)


# -----------------------------------------------------------------

In [ ]:
'''OPTIMIZE'''
# fair_fine_tune_model(DF_NCF,train_data, num_epochs, learning_rate,batch_size,num_negatives,n_careers,train_gender,fairness_thres,epsilonBase, unsqueeze=True)
emb_size = 128
num_epochs = 10
batch_size = 256

num_negatives = 5

random_samples = 15
top_k = 10

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(ncf.parameters(), lr=learning_rate, weight_decay=1e-6)

ncf.train()

torch.save(ncf.state_dict(), "models/DF_NCF")

# -----------------------------------------------------------------

In [ ]:
'''FAIR FINE TUNING MODEL'''
all_users = torch.LongTensor(train['uid'].values).to(device)
all_items = torch.LongTensor(train['job'].values).to(device)
# protected attribute
all_genders = torch.LongTensor(train['gender'].values).to(device)

from fairness_measures import Measures
m = Measures()

num_batches = np.int64(np.floor(train.shape[0] / batch_size))

for i in range(num_epochs):
    dataloader = DataLoader(sensitive, batch_size=batch_size,
                            shuffle=True, num_workers=0)
    t1 = time.time()

    it_per_epoch = len(data) / batch_size
    j = 1
    for batch in dataloader:
        u, j, g, r = batch
        # move tensors to cuda
        users = u.to(device)
        jobs = m.to(device)
        genders = g.to(device)
        ratings = r.to(device)
        
        # users = torch.LongTensor(batch_df.uid.to_numpy()).to(device)
        # items = torch.LongTensor(batch_df.job.to_numpy()).to(device)
        # ratings = torch.FloatTensor(batch_df.rating.to_numpy()).to(device)
        # print(items)
        y_hat = ncf(users, jobs)

        loss1 = nn.BCELoss(y_hat, ratings.unsqueeze(1))

        predicted_probs = ncf(all_users, all_items)
        avg_epsilon = m.computeEDF(all_genders,predicted_probs,n_careers,all_items,device)
        print(avg_epsilon)
        #criteroin hinge
        loss2 = torch.max(torch.tensor(0.0).to(device), (avg_epsilon-epsilonBase))

        loss = loss1 + fairness_thres*loss2

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'epoch: {i + 1} \nbatch: {j} out of: {num_batches} \naverage loss: {loss.item()}\n')
        j+=1

# -----------------------------------------------------------------

In [ ]:
torch.save(ncf.state_dict(), "models/DF_NCF")

# -----------------------------------------------------------------

In [ ]:
import math
import heapq
def evaluate_fine_tune(model,df_val,top_K,random_samples):
    model.eval()
    avg_HR = np.zeros((len(df_val),top_K))
    avg_NDCG = np.zeros((len(df_val),top_K))

    # for i in range(len(df_val)):
    test_df = sensitive.add_negatives(
        df_val,
        item='job',
        items=sensitive.jobs,
        n_samples=random_samples
    )
    users, items = torch.LongTensor(test_df.uid).to(device), torch.LongTensor(test_df.job).to(device)
    y_hat = model(users, items)

    y_hat = y_hat.cpu().detach().numpy().reshape((-1,))
    items = items.cpu().detach().numpy().reshape((-1,))
    map_item_score = {}
    for j in range(len(y_hat)):
        map_item_score[items[j]] = y_hat[j]
    for k in range(top_K):
        # Evaluate top rank list
        ranklist = heapq.nlargest(k, map_item_score, key=map_item_score.get)
        gtItem = items[0]
        avg_HR[i,k] = getHitRatio(ranklist, gtItem)
        avg_NDCG[i,k] = getNDCG(ranklist, gtItem)
    avg_HR = np.mean(avg_HR, axis = 0)
    avg_NDCG = np.mean(avg_NDCG, axis = 0)
    return avg_HR, avg_NDCG

def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

# -----------------------------------------------------------------

In [ ]:
'''EVALUATE TUNED MODEL'''
hr, ndcg = evaluate_fine_tune(ncf, test, top_K, random_samples)

hr

# -----------------------------------------------------------------

In [ ]:
'''MEASURE THE FAIRNESS OF THE MODEL'''
def fairness_measures(model,df_val,num_items):
    model.eval()
    users, items = torch.LongTensor(df_val.uid.to_numpy()).to(device), torch.LongTensor(df_val.job.to_numpy()).to(device)
    y_hat = model(users, items)

    avg_epsilon = m.computeEDF(all_genders.cpu(),y_hat,num_items,items,device)
    U_abs = m.compute_absolute_unfairness(all_genders.cpu(),y_hat,num_items,items,device)

    avg_epsilon = avg_epsilon.cpu().detach().numpy().reshape((-1,)).item()
    print(f"average differential fairness: {avg_epsilon: .3f}")

    U_abs = U_abs.cpu().detach().numpy().reshape((-1,)).item()
    print(f"absolute unfairness: {U_abs: .3f}")

fairness_measures(ncf, test, n_careers)

# -----------------------------------------------------------------